# Importação das bibliotecas necessárias

In [1]:
import pandas as pd 

from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer
from datetime import datetime
from sklearn.feature_selection import SelectPercentile
from scipy.stats import uniform


from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn import set_config
set_config(display='diagram')
import datetime
import time

# Importação dos dados

In [2]:
data = pd.read_csv('../dataset/processed/artigos_de_partidos/artigos_partidos.csv')

In [3]:
# remocao de dados nulos
data = data[data['Conteudo'] != '']

In [4]:
# remocao de colunas desnecessarias
rem_cols = ['URL']
data.drop(rem_cols, axis=1, inplace=True)

In [5]:
data.head() # visualização das primeiras 5 linhas do dataframe

,Partido,Conteudo,Vies
0,Novo,Multa imposta ao candidato na condenação foi...,direita
1,Novo,Cadastro será usado como identificação junt...,direita
2,Novo,A Bancada do NOVO na Câmara considera temerá...,direita
3,Novo,Um ambiente com ausência de segurança juríd...,direita
4,Novo,"Segundo o MP, o estado do RJ sequer utiliza os...",direita


In [6]:
# conversao dos rotulos categoricos para numericos
data['Vies'] = data['Vies'].map({'direita':2,
                                'centro': 1,
                                'esquerda': 0})

In [7]:
data.head()

,Partido,Conteudo,Vies
0,Novo,Multa imposta ao candidato na condenação foi...,2
1,Novo,Cadastro será usado como identificação junt...,2
2,Novo,A Bancada do NOVO na Câmara considera temerá...,2
3,Novo,Um ambiente com ausência de segurança juríd...,2
4,Novo,"Segundo o MP, o estado do RJ sequer utiliza os...",2


# Splits que serão avaliados

In [8]:
# a seguir os dados serão divididos entre features (X) e label (y)

X_columns = [column for column in data.columns if column != 'Vies']
X = data[X_columns] # features
X.head() 

,Partido,Conteudo
0,Novo,Multa imposta ao candidato na condenação foi...
1,Novo,Cadastro será usado como identificação junt...
2,Novo,A Bancada do NOVO na Câmara considera temerá...
3,Novo,Um ambiente com ausência de segurança juríd...
4,Novo,"Segundo o MP, o estado do RJ sequer utiliza os..."


In [9]:
y = data['Vies'] # label
y.head()

0    2
1    2
2    2
3    2
4    2
Name: Vies, dtype: int64

## Estratificação por viés

In [10]:
X_train_strat_vies, X_test_strat_vies, y_train_strat_vies, y_test_strat_vies = train_test_split(X, y.to_numpy(),
                                                                                                test_size=0.2,
                                                                                                random_state=42,
                                                                                                stratify=y)

X_train_strat_vies = X_train_strat_vies.drop('Partido', axis=1).Conteudo # remocao da coluna partido
X_test_strat_vies = X_test_strat_vies.drop('Partido', axis=1).Conteudo # remocao da coluna partido

In [11]:
X_train_strat_vies

3285     Alagoas – O prefeito de Maceió, JHC (PL-AL), ...
5772     Após Indicação e Ofício apresentados pelo ...
2306     Brasília – “Você importa. Escolha a vida!”. ...
1757     Nota do PCB Santa Catarina sobre a conquista d...
690      O NOVO foi fundado em 2011 por pessoas comuns ...
                               ...                        
3568     Amazonas – A deputada estadual Therezinha Ruiz...
4852     Brasília – O Ministério da Educação (MEC) ...
6461     O povo paraguaio está se insurgindo contra um...
10634    Na CCJ, governistas pedem vista e atrasam apre...
10822    PV Pernambuco repudia as ofertas do governo fe...
Name: Conteudo, Length: 9370, dtype: object

## Estratificação por partido

In [12]:
X_train_strat_part, X_test_strat_part, y_train_strat_part, y_test_strat_part = train_test_split(X, y.to_numpy(),
                                                                                                test_size=0.2,
                                                                                                random_state=42,
                                                                                                stratify=X['Partido'])

X_train_strat_part = X_train_strat_part.drop('Partido', axis=1).Conteudo # remocao da coluna partido
X_test_strat_part = X_test_strat_part.drop('Partido', axis=1).Conteudo # remocao da coluna partido

## Teste com partidos fora do treino

In [13]:
direita = data[data['Vies'] == 2] # selecao apenas dos partidos de direita
centro = data[data['Vies'] == 1] # selecao apenas dos partidos de centro
esquerda = data[data['Vies'] == 0] # selecao apenas dos partidos de esquerda
total = data.shape[0] # quantidade total de linhas no dataset

In [14]:
print('Porcentagem dos partidos de direita em relação ao dataset:')
for part in direita['Partido'].unique():
    
    qnt_part = direita[direita['Partido'] == part].shape[0]
    porc = qnt_part / total * 100
    print(f'porcentagem do partido {part}: {porc:.2f}%')

Porcentagem dos partidos de direita em relação ao dataset:
porcentagem do partido Novo: 10.22%
porcentagem do partido PL: 28.90%
porcentagem do partido PP: 5.01%
porcentagem do partido União Brasil: 2.42%


In [15]:
print('Porcentagem dos partidos de centro em relação ao dataset:')
for part in centro['Partido'].unique():
    
    qnt_part = centro[centro['Partido'] == part].shape[0]
    porc = qnt_part / total * 100
    print(f'porcentagem do partido {part}: {porc:.2f}%')

Porcentagem dos partidos de centro em relação ao dataset:
porcentagem do partido PDT: 3.82%
porcentagem do partido MDB: 5.13%
porcentagem do partido PSB: 15.04%
porcentagem do partido PV: 7.41%


In [16]:
print('Porcentagem dos partidos de esquerda em relação ao dataset:')
for part in esquerda['Partido'].unique():
    
    qnt_part = esquerda[esquerda['Partido'] == part].shape[0]
    porc = qnt_part / total * 100
    print(f'porcentagem do partido {part}: {porc:.2f}%')

Porcentagem dos partidos de esquerda em relação ao dataset:
porcentagem do partido PCB: 5.27%
porcentagem do partido PSOL: 0.13%
porcentagem do partido PSTU: 5.35%
porcentagem do partido PCDoB: 5.14%
porcentagem do partido PT: 5.10%
porcentagem do partido Rede: 1.06%


Baseado nas porcentagens acima, o partido de esquerda, centro e direita que foram escolhidos para o conjunto de teste representam, respectivamente 5.53%, 7.35% e 10.34% do dataset. Dessa forma, o conjunto de teste será constituído por 5.53 + 7.35 + 10.34 = 23.22% do dataset original.

In [17]:
part_teste = ['PSTU', 'PV', 'Novo'] # partidos do conjunto de teste

test = data[data['Partido'].isin(part_teste)].copy() # selecao dos dados de teste
test.drop('Partido', axis=1, inplace=True) # remocao da coluna partido

train = data[~data['Partido'].isin(part_teste)].copy() # selecao dos dados de treino
train.drop('Partido', axis=1, inplace=True) # remocao da coluna partido

In [18]:
X_train_part_novos = train.drop('Vies', axis=1).Conteudo # X_train
y_train_part_novos = train['Vies'].to_numpy() # y_train

X_test_part_novos = test.drop('Vies', axis=1).Conteudo # X_test
y_test_part_novos = test['Vies'].to_numpy() # y_test

# Seleção do modelo

In [19]:
# Defina a função para calcular a porcentagem com base no número total de features
def percentage_features(percentage, total_features):
    return int(np.ceil(percentage * total_features / 100.0))

In [20]:
def seleciona_grid_tfidf(model, split):

    param_grid = None
    
    if split == 'strat_vies':
        X_train = X_train_strat_vies.copy()
        
    elif split == 'strat_partido':
        X_train = X_train_strat_part.copy()
        
    elif split == 'pred_partido_novo':
        X_train = X_train_part_novos.copy()
        

    if isinstance(model, MultinomialNB):
            param_grid = {
            "vect__ngram_range": [(1,2), (1,3), (1,4), (2,3), (2,4), (3,4)],
            "vect__analyzer": ['word','char'],
            "selection__percentile": [33, 66, 100],
            "estimator__alpha": [50, 15, 10, 5, 1, 0.5, 0.3, 0.1, 0.05, 0.03, 0.02, 0.01,  0.001],
            "estimator__fit_prior": [True, False],
            }

    if isinstance(model, SVC):
            param_grid = {
            "vect__ngram_range": [(1,2), (1,3), (1,4), (2,3), (2,4), (3,4)],
            "vect__analyzer": ['word','char'],
            "selection__k": [200,400,600,800,1024],
            "estimator__gamma": [1, 0.1, 0.01, 0.001],
            "estimator__kernel": ['linear', 'sigmoid'],
            "estimator__C": [0.1, 1, 10, 100]
            }
            
    if isinstance(model, LinearSVC):
        
            param_grid = {
            "vect__ngram_range": [(1,2), (1,3), (1,4), (2,3), (2,4), (3,4)],
            "vect__analyzer": ['word','char'],
            "selection__percentile": [33, 66, 100],
            "estimator__dual": [True, False],
            "estimator__penalty": ['l1', 'l2'],
            "estimator__fit_intercept": [True, False],
            "estimator__C": uniform(loc=0, scale=4)
            }


    if isinstance(model, RandomForestClassifier):
        param_grid = {
        "vect__ngram_range": [(1,2), (1,3), (1,4), (2,3), (2,4), (3,4)],
        "vect__analyzer": ['word','char'],
        "selection__k": [200,400,600,800,1024],
        "estimator__n_estimators": np.arange(20,150), 
        "estimator__max_features": ['log2', 'sqrt'],
        "estimator__max_depth": np.arange(10,110),
        "estimator__min_samples_split": np.arange(2,11),
        "estimator__min_samples_leaf": np.arange(1,5),
        "estimator__bootstrap": [True, False]
        }
        
    if isinstance(model, XGBClassifier):
        param_grid = {
        "vect__ngram_range": [(1,2), (1,3), (1,4), (2,3), (2,4), (3,4)],
        "vect__analyzer": ['word','char'],
        "selection__percentile": [33, 66, 100],
        "estimator__gamma": np.linspace(0,9,100, dtype=np.int64),
        "estimator__alpha": np.linspace(0,40,100, dtype=np.int64),
        "estimator__lambda": np.linspace(0,3,10, dtype=np.int64),
        "estimator__colsample_bytree": np.linspace(0.2,1,10, dtype=np.int64)
        }

    return param_grid

In [21]:
def fit_e_avalia(split, random_search):
    
    inicio_random_search = datetime.datetime.now()
    
    if split == 'strat_vies': # estratificacao pela label
        model_trained = random_search.fit(X_train_strat_vies, y_train_strat_vies) # fit

        fim_random_search = datetime.datetime.now()
        tempo_total = fim_random_search - inicio_random_search
        print(f'Duração da Random Search: {tempo_total}')

        y_pred = model_trained.predict(X_test_strat_vies) # predicao
        f1 = f1_score(y_test_strat_vies, y_pred, average= 'macro') # f1
        report = classification_report(y_test_strat_vies, y_pred, output_dict=True) # class report

    elif split == 'strat_partido': # estratificacao pelos partidos 
        model_trained = random_search.fit(X_train_strat_part, y_train_strat_part) # fit

        fim_random_search = datetime.datetime.now()
        tempo_total = fim_random_search - inicio_random_search
        print(f'Duração da Random Search: {tempo_total}')

        y_pred = model_trained.predict(X_test_strat_part) # predicao
        f1 = f1_score(y_test_strat_part, y_pred, average= 'macro') # f1
        report = classification_report(y_test_strat_part, y_pred, output_dict=True) # class report

    elif split == 'pred_partido_novo': # predicao de partidos nao vistos no teste
        model_trained = random_search.fit(X_train_part_novos, y_train_part_novos) # fit

        fim_random_search = datetime.datetime.now()
        tempo_total = fim_random_search - inicio_random_search
        print(f'Duração da Random Search: {tempo_total}')

        y_pred = model_trained.predict(X_test_part_novos) # predicao
        f1 = f1_score(y_test_part_novos, y_pred, average= 'macro') # f1
        report = classification_report(y_test_part_novos, y_pred, output_dict=True) # class report
    
    return model_trained, tempo_total, f1, report

In [22]:
def compara_tfidf(iteracoes, modelos, nome_arquivo):

    # seletor de features
    selection = SelectPercentile()

    # possibilidades de oversampling ou nao
    samplers = [RandomOverSampler(random_state=42), None]

    # diferentes splits que serao avaliador
    splits = ['strat_vies', 'strat_partido', 'pred_partido_novo'] 
    
    # vetorizador do texto
    vectorizer = TfidfVectorizer()

    # dataframe em que sera inserido os dados do modelo testado 
    df_resultados = pd.DataFrame(columns=['modelo', 'split', 'sampler', 'scaling',
                                          'duracao_random_search','qnt_iteracoes',
                                          'f1_randsearch',
                                          'melhores_parametros', 'f1_pred',
                                          'class_report'])
    
    for model in modelos:

        for sampler in samplers:

            for split in splits:
                
                # seleciona grid de parametros
                param_grid = seleciona_grid_tfidf(model, split)
                
                # seleciona scaler
                if isinstance(model, MultinomialNB):
                    scaler = MaxAbsScaler() # normaliza e garante valores nao negativos
                else:
                    scaler = MaxAbsScaler()
        
                # define o pipeline
                pipeline = Pipeline([
                        ('vect', vectorizer), 
                        ('scaling', scaler), 
                        ('selection', selection),
                        ('ros', sampler),
                        ('estimator', model)
                        ])
        
                
                #  --- Prints das configurações dessa iteracao ---
                print(f'Modelo: {model}')
                print(f'Split: {split}')
                print(f'Scaler: {scaler}')
                print(f'Sampler: {sampler}')
                    
        
                # definicao da randomized search
                random_search = RandomizedSearchCV(pipeline, param_distributions=param_grid,cv=StratifiedKFold(n_splits=5),
                                                    n_iter=iteracoes, n_jobs=1, random_state=42, scoring='f1_macro')


                # fit e avaliacao pela randomized search
                model_trained, tempo_total, f1, report = fit_e_avalia(split, random_search)
                
                print('---')
                resultados = model_trained.cv_results_

                for params, score in zip(resultados['params'], resultados['mean_test_score']):
                    print(f"Parâmetros: {params}, Score: {score}")
                print('---')
                    
                # melhor metrica na random search
                score_random_search = model_trained.best_score_
                score_random_search *= 100
                score_random_search = round(score_random_search,2)
                print(f'Melhor F1 na Random Search: {score_random_search}%')
                
                # melhores parametros encontrados
                print('Melhores parâmetros encontrados:')
                print(model_trained.best_params_)

                
                # acuracia da predicao
                f1 *= 100
                f1 = round(f1,2)
                print(f'F1 macro = {f1}%')
        
                # classification report
                print(report)
                        
                
                print('----------------------------------------------')
                
                # --- Escrita em memória secundária ---

                # Nova linha que sera adicionada
                nova_linha = {'modelo': model, 'split': split,
                              'sampler': str(sampler), 'scaling': scaler,
                              'duracao_random_search': tempo_total,
                              'qnt_iteracoes': iteracoes,
                              'f1_randsearch': f'{score_random_search}%',
                              'melhores_parametros': str(model_trained.best_params_),
                              'f1_pred': f'{f1}%', 'class_report': report}
            
                # Cria um novo DataFrame com a nova linha
                nova_linha_resultados = pd.DataFrame([nova_linha])
            
                # Concatena o novo DataFrame com o DataFrame existente
                df_resultados = pd.concat([df_resultados, nova_linha_resultados], ignore_index=True)
    
    
    print('Fim dos testes')
    
    # salvamento do dataframe de resultados apos os testes terem terminado
    df_resultados.to_csv(nome_arquivo, index=False)

In [23]:
modelos = [MultinomialNB()]

compara_tfidf(40, modelos, 'compara-nb-tfidf.csv')

Modelo: MultinomialNB()
Split: strat_vies
Scaler: MaxAbsScaler()
Sampler: RandomOverSampler(random_state=42)
Duração da Random Search: 2:39:36.475990
---
Parâmetros: {'vect__ngram_range': (2, 3), 'vect__analyzer': 'char', 'selection__percentile': 100, 'estimator__fit_prior': True, 'estimator__alpha': 1}, Score: 0.802065947236831
Parâmetros: {'vect__ngram_range': (2, 4), 'vect__analyzer': 'char', 'selection__percentile': 100, 'estimator__fit_prior': False, 'estimator__alpha': 50}, Score: 0.8694765041714831
Parâmetros: {'vect__ngram_range': (3, 4), 'vect__analyzer': 'word', 'selection__percentile': 100, 'estimator__fit_prior': False, 'estimator__alpha': 10}, Score: 0.9443496096044679
Parâmetros: {'vect__ngram_range': (1, 4), 'vect__analyzer': 'char', 'selection__percentile': 33, 'estimator__fit_prior': True, 'estimator__alpha': 0.03}, Score: 0.8739083836592583
Parâmetros: {'vect__ngram_range': (1, 3), 'vect__analyzer': 'word', 'selection__percentile': 33, 'estimator__fit_prior': False, '